## Коллокации

Коллокации - это устойчивые выражения, состоящие из двух и более слов. Устойчивые - значит, что они часто используются вместе. Также часто значения коллокации не могут быть выведены лишь из значений, входящих в них слов.

Нахождение коллокаций на практике - это какая-то фильтрация нграммов в корпусе. Прежде чем, переходить к этому, вспомним как в питоне можно вытащить нграммы. Заодно повторим и нормализацию.

Возьмем какой-нибудь кусочек текста для примера.

In [67]:
text = """Доклад Андрея можно использовать как справочник и how-to по отладке проблем с утечкой нативной памяти на примере non-heap памяти. 

Доклад полезен для понимания, кто и что съедает память. Андрей показывает инструменты для анализа памяти, в том числе AsyncProfiler, который встроен в Idea: «The upcoming IntelliJ IDEA 2018.3 integrates a low overhead sampling profiler that can profile JVM and Native code – Async profiler».

Мы рекомендуем всем посмотреть его и прогнать по тем же шагам свои модули и микросервисы."""

### Задание 1. 

Напишите функцию, которая будет принимать текст, разбивать его на токены и группировать токены в нграммы. Параметр N - должен быть задаваемым.

In [68]:
def normalize(text):
    ### ваш код здесь
    return normalized_text


def ngrammer(text):
    tokens = normalize(text)
    ## ваш код здесь

Теперь перейдем к самим коллокациям.

## Данные 

Возьмём небольшую коллекцию научных статей с киберленинки. Они переведены в текст с помощью pdf2text, поэтому там полно всякого мусора.

In [36]:
import pandas as pd
import os
from collections import Counter, defaultdict
import numpy as np

In [2]:
data_files = ['ru_kw_eval_datasets-master/data/'+file for file in os.listdir('ru_kw_eval_datasets-master/data') \
                                           if 'cyber' in file and file.endswith('.jsonlines')]

In [3]:
data = pd.concat([pd.read_json(file, lines=True) for file in data_files[:2]], ignore_index=True)

In [4]:
pd.set_option('display.max_colwidth', 1000)

Преобразуем тексты в токены с помощью функции, которую написали.

In [5]:
data['content_norm'] = data['content'].apply(normalize)

Самый простой способ найти устойчивые сочетания - просто посчитать всё и взять самые частотные.

In [71]:
word_counter = Counter()

for text in data['content_norm'].values:
    word_counter.update(ngrammer(text))


Посмотрим, что получилось.

In [72]:
word_counter.most_common()

[(('of', 'the'), 1895),
 (('таким', 'образом'), 1413),
 (('рис', '1'), 1057),
 (('1', '2'), 1034),
 (('рис', '2'), 927),
 (('in', 'the'), 731),
 (('рис', '3'), 649),
 (('настоящее', 'время'), 604),
 (('2', '3'), 574),
 (('рис', '4'), 511),
 (('российской', 'федерации'), 503),
 (('список', 'литературы'), 482),
 (('1', '1'), 481),
 (('3', '4'), 464),
 (('точки', 'зрения'), 454),
 (('т', 'е'), 451),
 (('2', '2'), 449),
 (('литературы', '1'), 441),
 (('представляет', 'собой'), 412),
 (('рис', '5'), 385),
 (('4', '5'), 376),
 (('0', '0'), 375),
 (('т', 'д'), 375),
 (('et', 'al'), 366),
 (('литература', '1'), 355),
 (('табл', '1'), 349),
 (('научные', 'ведомости'), 335),
 (('томского', 'политехнического'), 333),
 (('политехнического', 'университета'), 333),
 (('on', 'the'), 332),
 (('5', '6'), 328),
 (('известия', 'томского'), 327),
 (('1', '−'), 325),
 (('−', '1'), 318),
 (('технические', 'науки'), 304),
 (('окружающей', 'среды'), 301),
 (('государственный', 'университет'), 295),
 (('таблиц

В списке много всяких чисел, однобуквеных слов и стоп-слов. 

### Задание 2.

Добавьте какие-нибудь ограничения к коду выше, чтобы биграммы получались почище.

В списке есть коллокации, которые попали в список из-за того, что одно слово очень частотное и вообще встречается много в каких контекстах. Нас скорее интересуют случаи, когда слова в большинстве случаев встречаются вместе. Для этого мы можем придумать какие-нибудь формулы, учитывающие частоты слов по отдельности и общую частоту.

In [86]:
## эта функция используется в работе Миколова про Word2Vec

def scorer(word_count_a, word_count_b, bigram_count, len_vocab, minimum_count):
    score = ((bigram_count - minimum_count) / (word_count_a * word_count_b))*len_vocab
    
    return score

In [122]:
def collect_stats(texts, scorer, threshold):
    ## соберем статистики для отдельных слов
    ## и биграммов
    
    word_counter = Counter()
    bigram_counter = Counter()
    
    for text in texts:
        word_counter.update(text)
        bigram_counter.update(ngrammer(text, 2))
    
    return word_counter, bigram_counter


def score_bigrams(words, bigrams, scorer, threshold):
    ## посчитаем метрику для каждого нграмма
    bigram2score = {}
    len_vobab = len(word_counter)
    for bigram in bigram_counter:
        score = scorer(word_counter[bigram[0]], word_counter[bigram[1]], 
                       bigram_counter[bigram], len_vobab, 5)
        
        ## если метрика выше порога, добавляем в словарик
        if score > threshold:
            bigram2score[bigram] = score
    
    return bigram2score

Посмотрим, что получилось.

In [114]:
bigram2score = score_bigrams(*collect_bigrams(data['content_norm'].values))

Можно применять расширить размер нграмма, а можно последовательно преобразовывать один и тот же текст, на каждом шагу собирая новые биграммы.

Напишием такую функцию.

In [46]:
def bigram_text(text, bigram2score):
    new_text = []
    i = 0
    while i < (len(text)-1):
        bigram = (text[i], text[i+1])
        if bigram in bigram2score:
            new_text.append('_'.join(bigram))
            i += 2
        else:
            new_text.append(text[i])
            i += 1
    else:
        if i == (len(text)-1):
            new_text.append(text[i])
    
    return new_text

Применим её.

In [115]:
data['content_norm_bigram'] = data['content_norm'].apply(lambda x: bigram_text(x, bigram2score))

In [116]:
trigram2score = collect_bigrams(data['content_norm_bigram'])

In [117]:
data['content_norm_trigram'] = data['content_norm_bigram'].apply(lambda x: bigram_text(x, trigram2score))

In [119]:
data['content_norm_trigram'] 

0                           [системный_анализ, удк, 519.865, 519.95, ббк, 22.165, одной, гипотезе, основаниях, теории, иерархических, игр, горелов, м, 1, вычислительный_центр, ран_москва, исследовании, игр, фиксированным, порядком, ходов, обычно, принимается, некоторая, гипотеза, поведении, игрока, нижнего_уровня, существенно, упрощающая, ответ, получающейся, задаче, рассуждения, приводящие, этому, ответу, ниже, обсуждается, адекватность, гипотезы, рассматривается, простейшая, игра, двух, лиц, примере, которой, видны, основные, проблемы, некоторые, подходы_решению, введение, пусть_имеется, игра, двух, лиц, которой, первый_игрок, выбирает, свои, управления, множества, u, второй, множества, v, будем_считать, цели, игроков, описываются, стремлением, максимизации, функций, g, h, соответственно, рассмотрим, следующую, схему, взаимодействия, игроков, традиции, носящей, имя, игры, γ2, 3, предположим, первый_игрок, рассчитывает, действительно, иметь, информацию, выборе, партнером, управления, 

In [60]:
sorted(bigram2score.items(), key=lambda x: x[1])

[(('2020', 'г'), 10.001272240088698),
 (('становление', 'рыночных'), 10.001373298304275),
 (('будут', 'учтены'), 10.001671891327064),
 (('единого', 'номера'), 10.001671891327064),
 (('трудной', 'ситуации'), 10.001671891327064),
 (('последнее', 'равенство'), 10.00216958598726),
 (('определенный', 'тип'), 10.002679774421217),
 (('социального', 'опыта'), 10.002819283015784),
 (('⎟', '≡'), 10.004135964224655),
 (('науки', 'интеллектуальное'), 10.004160860043799),
 (('управления', 'экономикой'), 10.004332870907051),
 (('населения', 'красноярского'), 10.004360031057756),
 (('элементов', 'конструкций'), 10.006205669086034),
 (('200', 'мпа'), 10.006584646224615),
 (('колебаний', 'амплитуды'), 10.00811464314055),
 (('оптимального', 'количества'), 10.012946144551048),
 (('замкнутый', 'объем'), 10.013330942132951),
 (('общественного', 'бытия'), 10.01484262778785),
 (('the', 'ﬁrst'), 10.0163996920386),
 (('determining', 'the'), 10.0163996920386),
 (('данной', 'проблеме'), 10.016690863546211),
 (('

Попробуем ещё одну функцию.

In [42]:
def scorer_pmi(word_count_a, word_count_b, bigram_count, corpus_size, minimum_count):
    score = ((bigram_count/corpus_size) / ((word_count_a/corpus_size) * (word_count_b/corpus_size)))
    
    return np.log(score) / -np.log((bigram_count/corpus_size))

In [57]:
def collect_stats(texts, scorer, threshold):
    ## соберем статистики для отдельных слов
    ## и биграммов
    
    word_counter = Counter()
    bigram_counter = Counter()
    
    for text in texts:
        word_counter.update(text)
        bigram_counter.update(ngrammer(text, 2))
    
    return word_counter, bigram_counter


def score_bigrams(words, bigrams, scorer, threshold):
    ## посчитаем метрику для каждого нграмма
    bigram2score = {}
    corpus_size = sum(word_counter.values())
    
    for bigram in bigram_counter:
        score = scorer(word_counter[bigram[0]], word_counter[bigram[1]], 
                       bigram_counter[bigram], corpus_size, 5)
        
        ## если метрика выше порога, добавляем в словарик
        if score > threshold:
            bigram2score[bigram] = score
    
    return bigram2score

In [58]:
bigram2score = collect_bigrams(data['content_norm'], scorer_pmi, -10000)

In [59]:
data['content_norm_bigram'] = data['content_norm'].apply(lambda x: bigram_text(x, bigram2score))

In [60]:
sorted(bigram2score.items(), key=lambda x: -x[1])

[(('6946', 'ulx-1'), 1.0000000000000002),
 (('pleurozium', 'schreberi'), 1.0000000000000002),
 (('індуктивному', 'накопичувачі'), 1.0),
 (('r1=1,2,…,ra', 'r2=1,2,…,rb'), 1.0),
 (('1818-7900', '¬âòúìëí'), 1.0),
 (('‘downward', 'causation’'), 1.0),
 (('0,95qu', '0,90qu'), 1.0),
 (('ðûáíîãî', 'õîçÿéñòâà'), 1.0),
 (('рогоза', 'широколистного'), 1.0),
 (('khon', 'v.ch'), 1.0),
 (('astroﬁz', 'issled'), 1.0),
 (('необъективному', 'судейству'), 1.0),
 (('дья', 'чарба'), 1.0),
 (('mya', 'arenaria'), 1.0),
 (('\x0e\x08', '\x04\x0f'), 1.0),
 (('hir', 'pozvonoc'), 1.0),
 (('\x0e\x0f\x10', '\x0e\x08'), 1.0),
 (('п669', 'п499'), 1.0),
 (('аббат', 'фоглер'), 1.0),
 (('cerastoderma', 'rusticum'), 1.0),
 (('communautés', 'européennes'), 1.0),
 (('четыреххлористом', 'углероде'), 1.0),
 (('åâê‰ûì‡ðó‰ìóâ', 'òóúðû‰ìë˜âòú‚ó'), 1.0),
 (('роговая', 'обманка'), 1.0),
 (('пжр', '2.200.26'), 1.0),
 (('1926-1938', '1939-1958'), 1.0),
 (('громилов', 'кинеловский'), 1.0),
 (('âñåðîññèéñêèé', 'íàó÷íî-èññëåäîâàòåëüñê

In [61]:
trigram2score = collect_bigrams(data['content_norm_bigram'], scorer_pmi, -10000)

In [62]:
data['content_norm_trigram'] = data['content_norm_bigram'].apply(lambda x: bigram_text(x, trigram2score))

In [63]:
data['content_norm_trigram'] 

0       [системный_анализ_удк, 519.865, 519.95, ббк, 22.165, одной, гипотезе, основаниях, теории, иерархических, игр, горелов_м, 1, вычислительный_центр, ран_москва, исследовании, игр, фиксированным, порядком, ходов, обычно_принимается, некоторая, гипотеза, поведении, игрока, нижнего_уровня, существенно, упрощающая, ответ, получающейся, задаче, рассуждения, приводящие, этому, ответу, ниже, обсуждается, адекватность, гипотезы, рассматривается, простейшая, игра, двух_лиц, примере, которой, видны, основные_проблемы, некоторые, подходы_решению, введение_пусть, имеется, игра, двух_лиц, которой, первый_игрок, выбирает, свои, управления, множества_u, второй, множества, v, будем_считать, цели, игроков, описываются, стремлением, максимизации, функций, g_h, соответственно, рассмотрим_следующую, схему, взаимодействия, игроков, традиции, носящей, имя, игры, γ2, 3, предположим, первый_игрок, рассчитывает, действительно, иметь, информацию, выборе, партнером, управления, v_∈, v, принятия, своего, око

In [64]:
bigram_text(data.iloc[0]['content_norm_trigram'], trigram2score)

['системный_анализ_удк',
 '519.865',
 '519.95',
 'ббк',
 '22.165',
 'одной',
 'гипотезе',
 'основаниях',
 'теории',
 'иерархических',
 'игр',
 'горелов_м',
 '1',
 'вычислительный_центр',
 'ран_москва',
 'исследовании',
 'игр',
 'фиксированным',
 'порядком',
 'ходов',
 'обычно_принимается',
 'некоторая',
 'гипотеза',
 'поведении',
 'игрока',
 'нижнего_уровня',
 'существенно',
 'упрощающая',
 'ответ',
 'получающейся',
 'задаче',
 'рассуждения',
 'приводящие',
 'этому',
 'ответу',
 'ниже',
 'обсуждается',
 'адекватность',
 'гипотезы',
 'рассматривается',
 'простейшая',
 'игра',
 'двух_лиц',
 'примере',
 'которой',
 'видны',
 'основные_проблемы',
 'некоторые',
 'подходы_решению',
 'введение_пусть',
 'имеется',
 'игра',
 'двух_лиц',
 'которой',
 'первый_игрок',
 'выбирает',
 'свои',
 'управления',
 'множества_u',
 'второй',
 'множества',
 'v',
 'будем_считать',
 'цели',
 'игроков',
 'описываются',
 'стремлением',
 'максимизации',
 'функций',
 'g_h',
 'соответственно',
 'рассмотрим_следующую

In [71]:
def scorer_ttest(word_count_a, word_count_b, bigram_count, corpus_size, minimum_count):
    mu = ((word_count_a/corpus_size) * (word_count_b/corpus_size))
    x_ = (bigram_count/corpus_size)
    score = (x_ - mu) / np.sqrt(x_/corpus_size)
    
    return score

In [74]:
bigram2score = collect_bigrams(data['content_norm'], scorer_ttest, 2.5)

In [75]:
sorted(bigram2score.items(), key=lambda x: -x[1])

[(('of', 'the'), 86.46798953478357),
 (('таким', 'образом'), 74.0332394797244),
 (('рис', '1'), 64.03492838375335),
 (('рис', '2'), 60.16406711596151),
 (('1', '2'), 58.18611577155665),
 (('in', 'the'), 55.395972829712875),
 (('рис', '3'), 51.34621630850245),
 (('настоящее', 'время'), 48.196759198057116),
 (('0', '0'), 47.748609734641946),
 (('2', '3'), 45.61193907653131),
 (('т', 'е'), 44.787558136374756),
 (('список', 'литературы'), 44.30476499673534),
 (('рис', '4'), 43.43944238006089),
 (('точки', 'зрения'), 42.35651082910067),
 (('литературы', '1'), 42.2439944724295),
 (('3', '4'), 41.65977900643294),
 (('российской', 'федерации'), 41.04429206168445),
 (('представляет', 'собой'), 40.16934253093707),
 (('et', 'al'), 39.81390974356149),
 (('4', '5'), 38.025837264377635),
 (('томского', 'политехнического'), 37.92577250637692),
 (('известия', 'томского'), 37.78151685219608),
 (('политехнического', 'университета'), 37.66246129534287),
 (('on', 'the'), 36.952603451993106),
 (('т', 'д'),

In [77]:
def collect_stats(texts):
    word_counter = Counter()
    bigram_counter = Counter()
    
    for text in texts:
        word_counter.update(text)
        bigram_counter.update(zip(text[:-1], text[1:]))
    return word_counter, bigram_counter

In [78]:
words, bigrams = collect_stats(data['content_norm'])

In [20]:



def collect_bigrams(words, bigrams, scorer):
    corpus_size = sum(bigrams.values())
    bigram2score = {}
    for bigram in bigrams:
        a = bigrams[bigram]
        b = words[bigram[0]] - a
        c = words[bigram[1]] - a
        d = corpus_size - b - c - a

        a = a/corpus_size
        b = b/corpus_size
        c = c/corpus_size
        d = d/corpus_size
    
        bigram2score[bigram] = scorer(a,b,c,d)
    
    return bigram2score
    

In [16]:
def jaccard(a,b,c,d):
    return a/(a+b+c)

In [89]:
bigram2score = collect_bigrams(words, bigrams, jaccard)

In [90]:
sorted(bigram2score.items(), key=lambda x: -x[1])

[(('еm2:=4', 'f1:=1000'), 1.0),
 (('4643', '5916'), 1.0),
 (('ɋɋɫɮɜɩɩ', 'ɋɏȼɉɉ'), 1.0),
 (('br11', 'r11dϕ'), 1.0),
 (('191,80', '182,64'), 1.0),
 (('implant', '1‑year'), 1.0),
 (('изомеризацию', 'дегидроциклизацию'), 1.0),
 (('c–n', '66,24'), 1.0),
 (('iarm', '178b'), 1.0),
 (('албэчрэ', 'дачий'), 1.0),
 (('ladur@micran.ru', 'ladur'), 1.0),
 (('кель', 'ватская'), 1.0),
 (('полі', '‘ничего'), 1.0),
 (('старополтавский', 'урюпинский'), 1.0),
 (('разввитии', 'трансгграничных'), 1.0),
 (('600410', '610610'), 1.0),
 (('круглящиеся', 'трапециевидные'), 1.0),
 (('bogoslovskomu', 'rassmotreniyu'), 1.0),
 (('матине', 'дафтари'), 1.0),
 (('ионно\x08катионных', 'подрешет\x08'), 1.0),
 (('aspidiotus', 'nerii'), 1.0),
 (('eso548−025', 'j032900.7−220848'), 1.0),
 (('0.477±', '0.207'), 1.0),
 (('с70fy', 'c76f54'), 1.0),
 (('ɩɪɢɪɨɞɨɨɯɪɚɧɧɵɟ', 'ɫɬɨɢɦɨɫɬɧɵɟ'), 1.0),
 (('105837.45+401355.7', '176.459+63.887'), 1.0),
 (('0.22073', '0.04413'), 1.0),
 (('неправдивым', 'лживым'), 1.0),
 (('lisovcova', 'anast

По этой ссылке можно прочитать про разные метрики.

http://www.scielo.org.mx/scielo.php?script=sci_arttext&pid=S1405-55462016000300327#t1

In [41]:
def get_window_stats(texts, window):
    
    bigrams = defaultdict(list)
    for text in texts:
        for i in range(len(text)-window):
            words = list(enumerate(text[i:i+window]))
            target = words[0][1]
            for j, word in words[1:]:
                bigrams[(target, word)].append(j)
    bigrams_stds = {}
    for bigram in bigrams:
        if len(bigrams[bigram]) > 5:
            bigrams_stds[bigram] = np.std(bigrams[bigram])
    return bigrams_stds

In [42]:
bigrams = get_window_stats(data['content_norm'], 3)

In [43]:
len(bigrams)

41751

In [28]:
bigram2score = collect_bigrams(words, bigrams, jaccard)

In [46]:
sorted(bigrams.items(), key=lambda x: x[1])

[(('характерные', 'черты'), 0.0),
 (('2(42', '6'), 0.0),
 (('контактов', 'ѕ'), 0.0),
 (('становится', 'одним'), 0.0),
 (('the', 'experimental'), 0.0),
 (('образовательные', 'программы'), 0.0),
 (('поэтому', 'время'), 0.0),
 (('®', 'min'), 0.0),
 (('основные', 'источники'), 0.0),
 (('имеет', 'большую'), 0.0),
 (('университета', 'науки'), 0.0),
 (('исполнительного', 'ур'), 0.0),
 (('высокой', 'скоростью'), 0.0),
 (('подготовке', 'архиерейскому'), 0.0),
 (('частоте', 'мспи'), 0.0),
 (('коммунального', 'теплоснабжения'), 0.0),
 (('вклад', 'вузовского'), 0.0),
 (('научного', 'качества'), 0.0),
 (('показан', '3'), 0.0),
 (('а.и', 'а.а'), 0.0),
 (('а.в', 'в.а'), 0.0),
 (('дрожжевых', 'микроорганизмов'), 0.0),
 (('белгород', 'ул'), 0.0),
 (('поскольку', 'именно'), 0.0),
 (('вымороженной', 'воды'), 0.0),
 (('распределении', 'тормозных'), 0.0),
 (('практическое', 'м'), 0.0),
 (('учащихся', 'классов'), 0.0),
 (('дизайн', 'среды'), 0.0),
 (('8', 'следует'), 0.0),
 (('территориального', 'развития')

In [30]:
def collect_bigrams(words, bigrams, scorer, threshold):
    bigram2score = {}
    
    corpus_size = sum(words.values())
    
    for bigram in bigrams:
        if bigrams[bigram] > 5:
            
            score = scorer(words[bigram[0]], words[bigram[1]], 
                       bigrams[bigram], corpus_size, 5)
            if score > threshold:
                bigram2score[bigram] = score
    
    return bigram2score

In [31]:
def scorer_pmi(word_count_a, word_count_b, bigram_count, corpus_size, minimum_count):
    score = ((bigram_count/corpus_size) / ((word_count_a/corpus_size) * (word_count_b/corpus_size)))
    
    return np.log(score) / -np.log((bigram_count/corpus_size))

In [32]:
bigram2score = collect_bigrams(words, bigrams, scorer_pmi, -1000)

In [33]:
sorted(bigram2score.items(), key=lambda x: -x[1])

[(('инв', 'инв'), 1.079549904062734),
 (('нетокс', 'нетокс'), 1.0740576493490204),
 (('◊', '◊'), 1.0739645839237788),
 (('0.0005', '0.0005'), 1.065360829522584),
 (('wc-8co-tac', 'wc-8co-tac'), 1.065360829522584),
 (('216,29', '216,29'), 1.065360829522584),
 (('add', 'add'), 1.0641885410658882),
 (('1,145', '1,145'), 1.059700561808594),
 (('wc-8co-vc', 'wc-8co-vc'), 1.053162352295881),
 (('казнить', 'миловать'), 1.0523362337668203),
 (('814', '814'), 1.0521437489051424),
 (('55,5', '55,5'), 1.0388917314089463),
 (('57,0', '57,0'), 1.0343144336216308),
 (('∂w', '∂w'), 1.0311292859126453),
 (('59,5', '59,5'), 1.0294774917258074),
 (('58,0', '58,0'), 1.0265788527023807),
 (('10-5', '10-5'), 1.0243556877973616),
 (('гол', 'гол'), 1.021322788989427),
 (('92,9', '92,9'), 1.021322788989427),
 (('55,8', '55,8'), 1.0190785442937018),
 (('íàóê', 'äîöåíò'), 1.0150439096618031),
 (('0123', '7654'), 1.0142300410309135),
 (('58,5', '58,5'), 1.0112571276894353),
 (('сетецентрическое', 'многоагентные'

Удобно пользоваться phraser из gensim'а. Он собирает статистику по корпусу, а затем склеивает слова в биграммы. Так как мы сделали выше. Если не нравятся функции оценки, то ему можно подать любую другую функцию.

In [48]:
import gensim

In [50]:
ph = gensim.models.Phrases(data['content_norm'])
p = gensim.models.phrases.Phraser(ph)

In [52]:
ph2 = gensim.models.Phrases(p[data['content_norm']])
p2 = gensim.models.phrases.Phraser(ph2)

In [53]:
p2[p[data['content_norm'].iloc[1]]]

['4',
 'civil_securitytechnology_vol_8',
 '2011_no_2',
 '28',
 'удк',
 '621.039.586',
 'научно-методическое',
 'сопровождение',
 'работ',
 'ликвидации_последствий',
 'аварии_чернобыльской_аэс',
 'issn_1996-8493_©_технологии',
 'гражданской_безопасности',
 '2011',
 'б.а',
 'галушкин',
 'с.в',
 'горбунов',
 'в.с',
 'исаев',
 'в.н',
 'лисица',
 'в.ф',
 'митрофанов',
 'с.п',
 'тодосейчук',
 'аннотация_статье',
 'кратко',
 'изложена',
 'деятельность',
 'сотрудников',
 'всесоюзного',
 'научно-исследовательского_института',
 'гражданской_обороны',
 'ликвидации_последствий',
 'аварии_чернобыльской_аэс',
 'scientific_and',
 'methodic',
 'support',
 'of',
 'works',
 'on',
 'liquidation',
 'of',
 'consequences_of',
 'the',
 'accident',
 'on',
 'chernobyl',
 'nuclear',
 'power',
 'station',
 'issn_1996-8493_©_civil',
 'security_technology',
 '2011',
 'b',
 'galushkin',
 's',
 'gorbunov',
 'v',
 'isaev',
 'v',
 'lisitsa',
 'v',
 'mitrofanov',
 's',
 'todosejchuk',
 'abstract',
 'article_shows',
 't

In [54]:
import nltk
from nltk.collocations import *

In [55]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [61]:
finder2 = BigramCollocationFinder.from_documents(data['content_norm'])

In [63]:
finder3 = TrigramCollocationFinder.from_documents(data['content_norm'])

In [64]:
finder2.nbest(bigram_measures.raw_freq, 100)

[('of', 'the'),
 ('таким', 'образом'),
 ('рис', '1'),
 ('1', '2'),
 ('рис', '2'),
 ('in', 'the'),
 ('рис', '3'),
 ('настоящее', 'время'),
 ('2', '3'),
 ('рис', '4'),
 ('российской', 'федерации'),
 ('список', 'литературы'),
 ('1', '1'),
 ('3', '4'),
 ('точки', 'зрения'),
 ('т', 'е'),
 ('2', '2'),
 ('литературы', '1'),
 ('представляет', 'собой'),
 ('рис', '5'),
 ('4', '5'),
 ('0', '0'),
 ('т', 'д'),
 ('et', 'al'),
 ('литература', '1'),
 ('табл', '1'),
 ('научные', 'ведомости'),
 ('политехнического', 'университета'),
 ('томского', 'политехнического'),
 ('on', 'the'),
 ('5', '6'),
 ('известия', 'томского'),
 ('1', '−'),
 ('−', '1'),
 ('технические', 'науки'),
 ('окружающей', 'среды'),
 ('государственный', 'университет'),
 ('таблица', '1'),
 ('астрофизический', 'бюллетень'),
 ('2', '1'),
 ('2008', 'г'),
 ('первую', 'очередь'),
 ('the', 'article'),
 ('i', '1'),
 ('2009', 'г'),
 ('следует', 'отметить'),
 ('рис', '6'),
 ('to', 'the'),
 ('of', 'a'),
 ('ведомости', 'серия'),
 ('6', '7'),
 ('0', 

In [65]:
finder3.nbest(trigram_measures.pmi, 100)

[('\x01m/σr', '∆m/[σ', '\u202b׀\u202cn-β\u202b]׀\u202c'),
 ('\x02\x07', '\x04\x0e', '\x07)\x0f'),
 ('\x02\x07\x10\x02\x11',
  '\x02\x12\x03\x05\x07\x07\x13\x0f\x14\x05\x03\x0f\x11',
  '\x01\x02\x03\x04\x05\x06\x07'),
 ('\x02\x0e', '\x16\x17\x0f\x14\x18', '\x02\x04\x19\x07\x19\x17\x11\x02\x05'),
 ('\x07\x13\x0e\x05\x14', '\x0f\x10', '\x06\x02'),
 ('\x07\x15\x04', '\x02\x07', '\x04\x0e'),
 ('\x07ieee', '802.15', 'wpan'),
 ('\x08\x0e\x0f', '\x04\x03', '\x07\x06\x08'),
 ('\x08\x12\x13\x06', '\x04\x03\x14', '\x0e\x15\x10'),
 ('\x0f\x07\x19\x17\x19\x02\x0f',
  "\x0e\x07\x17%\x07'\x02\x07\x07\x07\x03\x0f\x11\x07\x04\x02\x05\x03",
  '\x12\x02\x07'),
 ('\x0f\x10', '\x06\x02', '\x11\x12\x13\x14\x02\x04'),
 ('\x10\x03-\x07\x03\x0f\x11\x07\x10',
  '\x0f\x07\x19\x17\x19\x02\x0f',
  "\x0e\x07\x17%\x07'\x02\x07\x07\x07\x03\x0f\x11\x07\x04\x02\x05\x03"),
 ('\x10k', 'o\x15', 'z\x14'),
 ('\x10sr3', 'sc11', 'ra11'),
 ('\x14\x14\x15\x12\x03\x04\x08\x06\x0f',
  '\x10\x04\x08\x06\x16\x12',
  '\x10\x03\x06')